In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
pip install beautifulsoup4

In [ ]:
pip install fake_useragent

In [ ]:
pip install requests

Downloading urls in csv file

In [ ]:
import csv
import json
import hashlib
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from fake_useragent import UserAgent

# Define the root URL
root_url = 'https://www.jagran.com/fact-check/news-news-hindi.html'

# Function to extract article URL
def extract_article_data(article):
    link = article.find('a')['href']
    article_url = urljoin(root_url, link)
    return {
        'URL': article_url
    }

# Function to scroll to the bottom of the page
def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Reduced sleep duration

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--disable-images")

ua = UserAgent()
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')

# Run Chrome in headless mode to avoid graphical interface issues
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Start ChromeDriver with options
driver = webdriver.Chrome(options=options)

# Load the root URL
driver.get(root_url)

# Create an empty list to store article URLs
article_urls = []
unique_urls = set()

# Set the target number of articles
target_articles = 5000

# Load more until the target number of articles is reached or no more articles to load
while len(unique_urls) < target_articles:
    try:
        # Extract URLs from the current page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.select('article')

        for article in articles:
            try:
                article_info = extract_article_data(article)
                article_hash = hashlib.sha256(json.dumps(article_info, sort_keys=True).encode()).hexdigest()
                if article_hash not in unique_urls:
                    unique_urls.add(article_hash)
                    article_urls.append(article_info)
                    print("URL '{}' has been processed".format(article_info['URL']))
            except Exception as e:
                print(f"An error occurred while processing an article: {str(e)}")

        # Execute JavaScript to click the 'Load More' button
        load_more_button = driver.find_elements(By.XPATH, "//button[contains(@class, 'ListingSide_btn') and text()='Load More']")
        if load_more_button:
            driver.execute_script("arguments[0].click();", load_more_button[0])
            scroll_to_bottom(driver)
        else:
            print("No more articles to load.")
            break
    except StaleElementReferenceException:
        print("StaleElementReferenceException. Retrying...")
        time.sleep(2)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        break

# Close the WebDriver
driver.quit()

# Display the number of unique URLs
num_unique_urls = len(article_urls)
print(f"Number of unique URLs: {num_unique_urls}")

# Write the article URLs to a CSV file
output_file_path = "article_urls 1300.csv"
with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for article_info in article_urls:
        writer.writerow(article_info)



Removing url that contain, "sach-ke-saathi"

In [ ]:
import csv
import re

def filter_urls(csv_file, column_index):
    filtered_urls = []
    with open(csv_file, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            url = row[column_index]
            if not re.search(r'sach-', url):
                filtered_urls.append(row)
    return filtered_urls

def write_filtered_csv(filtered_data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(filtered_data)

# Replace 'input.csv' and 'output.csv' with your file paths
input_csv = "/content/article_urls 1300.csv"  # Your input CSV file
output_csv = "output 1293.csv"  # Output CSV file after filtering
url_column_index = 0  # Index of the column containing URLs

filtered_data = filter_urls(input_csv, url_column_index)
write_filtered_csv(filtered_data, output_csv)

print("Done")


Done


Adding 'Extract URL' column to retrieve the full news from Vishwasnews.

In [ ]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

def extract_particular_links(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            particular_links = [link.get('href') for link in soup.find_all('a') if link.get('href') and link.get('href').startswith('https://www.vishvasnews.com/')]
            return particular_links
        else:
            print(f"Failed to fetch webpage {url}. Status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"Error occurred while processing webpage {url}: {e}")
        return []

def process_csv(input_csv_file, output_csv_file):
    total_links = 0
    unique_links = set()

    # Read input CSV file
    df = pd.read_csv(input_csv_file)

    # Extract URLs from 'Article_URL' column
    urls = df['Article_URL'].tolist()

    # Extract particular links for each URL using multithreading
    with ThreadPoolExecutor() as executor:
        results = executor.map(extract_particular_links, urls)

    # Collect all unique links
    for extracted_links in results:
        for link in extracted_links:
            unique_links.add(link)

    # Write original columns along with the extracted URLs to the output CSV file
    with open(output_csv_file, 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(df.columns.tolist() + ["Extracted_URL"])  # Write column headers

        for idx, row in df.iterrows():
            article_url = row['Article_URL']
            extracted_links = extract_particular_links(article_url)
            for link in extracted_links:
                if link in unique_links:
                    writer.writerow(row.tolist() + [link])
                    unique_links.remove(link)
                    total_links += 1

    print(f"Total number of extracted unique links: {total_links}")

# Provide the path to your input CSV file containing the URLs and the desired output CSV file
input_csv_file = '/content/downloaded_images 1293.csv'
output_csv_file = 'output_data New Not Repeat 2.csv'

# Run the function to process CSV and extract URLs
process_csv(input_csv_file, output_csv_file)


Total number of extracted unique links: 1138


To run after the above one, so that as it adds the column "Text under 'Fact Check'" to it

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

def extract_content_from_url(row):
    url = row['Extracted_URL']
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            claim_review_div = soup.find('div', class_='reviews')
            if claim_review_div:
                claim_review_ul = claim_review_div.find('ul', class_='claim-review')
                if claim_review_ul:
                    extracted_texts = []
                    for li in claim_review_ul.find_all('li'):
                        strong = li.find('strong', text="Fact Check :")
                        if strong:
                            span = li.find('span')
                            if span:
                                extracted_texts.append(span.text.strip())
                    row['Text under Fact Check'] = ', '.join(extracted_texts) if extracted_texts else "No text found under 'Fact Check'"
                    return row
            row['Text under Fact Check'] = "No 'Fact Check' section found"
            return row
        else:
            print(f"Failed to fetch webpage {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred while processing webpage {url}: {e}")
        return None

def process_csv(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        rows = [row for row in reader]

    with ThreadPoolExecutor() as executor:
        results = executor.map(extract_content_from_url, rows)

    with open(output_csv_file, 'w', newline='') as output_file:
        fieldnames = list(rows[0].keys()) + ['Text under Fact Check'] if 'Text under Fact Check' not in rows[0].keys() else list(rows[0].keys())
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        for result in results:
            if result:
                writer.writerow(result)

# Provide the path to your CSV file containing the URLs and the desired output CSV file
input_csv_file = '/content/output_data New Not Repeat 2.csv'
output_csv_file = 'output Text Fact Type.csv'

# Run the function to process CSV and extract content
process_csv(input_csv_file, output_csv_file)


Adding Title to above csv

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

def extract_title_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            title_tag = soup.find('h1', class_='article-heading')
            if title_tag:
                return title_tag.text.strip()
            else:
                return "Title not found"
        else:
            print(f"Failed to fetch webpage {url}. Status code: {response.status_code}")
            return "Title not found"
    except Exception as e:
        print(f"Error occurred while processing webpage {url}: {e}")
        return "Title not found"

def process_csv(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        rows = [row for row in reader]

    titles = []
    with ThreadPoolExecutor() as executor:
        results = executor.map(lambda row: extract_title_from_url(row['Extracted_URL']), rows)
        titles.extend(results)

    for i, row in enumerate(rows):
        row['Title'] = titles[i]

    with open(output_csv_file, 'w', newline='') as output_file:
        fieldnames = list(rows[0].keys())
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

# Provide the path to your CSV file containing the URLs and the desired output CSV file
input_csv_file = '/content/output Text Fact Type.csv'
output_csv_file = 'output_file_with_title.csv'

# Run the function to process CSV and extract content
process_csv(input_csv_file, output_csv_file)


Adding Description

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

def extract_description_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            main_article_div = soup.find('div', class_='main_article_area')
            if main_article_div:
                description_paragraphs = main_article_div.find_all('p')
                description_text = ' '.join([p.text.strip() for p in description_paragraphs])
                return description_text.strip()
            else:
                return "Description not found"
        else:
            print(f"Failed to fetch webpage {url}. Status code: {response.status_code}")
            return "Description not found"
    except Exception as e:
        print(f"Error occurred while processing webpage {url}: {e}")
        return "Description not found"

def process_csv(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames + ['Description'] if 'Description' not in reader.fieldnames else reader.fieldnames
        rows = [row for row in reader]

    with ThreadPoolExecutor() as executor:
        for row in rows:
            url = row['Extracted_URL']
            description = extract_description_from_url(url)
            row['Description'] = description

    with open(output_csv_file, 'w', newline='') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

# Provide the path to your CSV file containing the URLs and the desired output CSV file
input_csv_file = '/content/output_file_with_title.csv'
output_csv_file = 'output_file_with_description.csv'

# Run the function to process CSV and extract content
process_csv(input_csv_file, output_csv_file)


Dropping rows that contain :"No 'Fact Check' section found"]

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("/output_file_with_description N.csv")

# Filter out rows where the "Text under Fact Check" column contains "No 'Fact Check' section found"
df_filtered = df[df['Text under Fact Check'] != "No 'Fact Check' section found"]

# Save the filtered DataFrame to a new CSV file
df_filtered.to_csv("filtered_file.csv", index=False)
